In [1]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
folder = "env"
sourcedir = currentdir.split(folder)[0]
sys.path.insert(0, sourcedir)
# from env.equation.equation import Equation
print(sourcedir)


/home/valdecar/Documents/projects/project/parser/


In [4]:
from env.equation.equation import Equation
from translator.tokenizer.tokenizer_main import LexNetTokenizer
from env.equation.data.terms.input.wolfram.lex_net_wolfram import LexNetW

from translator.tree.maps import map_tree, map_tree_postproc


Для преобразования термов wolfram в cpp можно было использовать обычный лексический анализ с помощью re: 

In [6]:

eq = Equation("U-c*D[U,{x,2}]")

tokenizer = LexNetTokenizer(LexNetW())

# make tokenization:
lexem = tokenizer.lex(eq.sent)

# convert lexem for cpp replacer:
# for replacer One need nodes object
# instead of Word. So need a tree.
# parser.parse() create tree but
# use it like list
eq.parser.parse()
lexem = eq.eq_tree

eq.replacer.cpp.editor.set_default()

# replace:
# just use nodes in tree like in list
cpp_map = [map_tree(term, eq.replacer.cpp.gen)
           for term in lexem]
out = [map_tree_postproc(term, eq.replacer.cpp.gen)
       for term in cpp_map]

print([o.output.cpp.out for o in out])

['-', '*', '(DXM2 * (source[delay][idx + 1 * Block0StrideX * Block0CELLSIZE + 0] - 2.0 * source[delay][idx + 0 * Block0StrideX * Block0CELLSIZE + 0] + source[delay][idx-1 * Block0StrideX * Block0CELLSIZE + 0]))', 'params[2]', 'source[delay][idx + 0]']


Однако, некоторые термы не потдаются лексическому разбору, например скобочные (функции, pow). Так, в следующем примере неопределенность к какому терму принадлежит скобка "(" перед U. Поэтому возникает необходимость использовать синтаксический анализ (восходящий разбор с использованием алгоритма Кока — Янгера — Касами https://en.wikipedia.org/wiki/CYK_algorithm) 

In [7]:
eq_pow = Equation("U'=(V(t-3.1)+(U+V))^3")
eq_pow.parser.parse()
eq_pow.replacer.cpp.editor.set_default()
eq_pow.replacer.cpp.map_cpp()
eq_pow.replacer.cpp.show_cpp()

source[delay][idx + 0]=pow(source[delay][idx + 1]+(source[delay][idx + 0]+source[delay][idx + 1]),3)
